In [2]:
import pandas as pd

# Read the merged_player_data.csv file
data = pd.read_csv('merged_player_data.csv')

# Calculate the Points_Percent_Diff column and round to five decimal places
data['Points_Percent_Diff'] = round(data['Points_Diff'] / data['Points_Running_Avg'], 5)

# Calculate the Rebounds_Percent_Diff column and round to five decimal places
data['Rebounds_Percent_Diff'] = round(data['Rebounds_Diff'] / data['Rebounds_Running_Avg'], 5)

# Calculate the Assists_Percent_Diff column and round to five decimal places
data['Assists_Percent_Diff'] = round(data['Assists_Diff'] / data['Assists_Running_Avg'], 5)

# Write the updated dataframe to percent_diff.csv
data.to_csv('percent_diff.csv', index=False)


In [13]:
# Read the merged_player_data.csv file
data = pd.read_csv('merged_player_data.csv')

# Group the data by Player column
grouped_data = data.groupby('Player')

# Calculate the standard deviation of Points_Running_Avg, Rebounds_Running_Avg, and Assists_Running_Avg
std_points = round(grouped_data['PTS'].std(), 5)
std_rebounds = round(grouped_data['TRB'].std(), 5)
std_assists = round(grouped_data['AST'].std(), 5)

# Create new columns for the standard deviation of each
data['Points_Running_Avg_Std'] = data['Player'].map(std_points)
data['Rebounds_Running_Avg_Std'] = data['Player'].map(std_rebounds)
data['Assists_Running_Avg_Std'] = data['Player'].map(std_assists)

# Write the updated dataframe to percent_diff.csv
data.to_csv('stdev.csv', index=False)



In [14]:

# Read the stdev.csv file
data = pd.read_csv('stdev.csv')

# Calculate the z-score for PTS
data['PTS_z'] = round((data['PTS'] - data['Points_Running_Avg']) / data['Points_Running_Avg_Std'], 5)

# Calculate the z-score for TRB
data['TRB_z'] = round((data['TRB'] - data['Rebounds_Running_Avg']) / data['Rebounds_Running_Avg_Std'], 5)

# Calculate the z-score for AST
data['AST_z'] = round((data['AST'] - data['Assists_Running_Avg']) / data['Assists_Running_Avg_Std'], 5)

# Write the updated dataframe to z_score.csv
data.to_csv('z_score.csv', index=False)


In [15]:
import numpy as np

In [16]:
# Read the z_score.csv file
data = pd.read_csv('z_score.csv')

# Add columns for bet results
data['PTS_Bet_Res'] = np.where(data['PTS'] > data['player_points_line'], 'Over', 'Under')
data['TRB_Bet_Res'] = np.where(data['TRB'] > data['player_rebounds_line'], 'Over', 'Under')
data['AST_Bet_Res'] = np.where(data['AST'] > data['player_assists_line'], 'Over', 'Under')

# Write the updated dataframe to bet_result.csv
data.to_csv('bet_result.csv', index=False)



In [2]:
import pandas as pd
import numpy as np

In [32]:


# Read the bet_result.csv file
data = pd.read_csv('bet_result.csv')


# Loop through the range of z-scores from 0.5 to 2.0 in increments of 0.1
for z_thresh in np.arange(0.5, 2.0, 0.1):
    # Add the prev_performance column based on the conditions and values
    data['pred_prev'] = np.where(data['PTS_z'] >= z_thresh, "Under", "NA")

    # Shift the values of pred_prev down by 1
    data['pred_prev'] = data['pred_prev'].shift(1)

    # Find the row with the smallest G for each player
    min_G_rows = data.groupby('Player')['G'].idxmin()

    # Replace the prev_pred in the rows with the smallest G with "NA"
    data.loc[min_G_rows, 'pred_prev'] = "NA"


    # Count the number of rows where pred_prev is not "NA" and is equal to PTS_Bet_Res
    success_count = len(data[(data['pred_prev'] == 'Under') & (data['PTS_Bet_Res'] == 'Under')])

    count_pred_prev_not_na = len(data[data['pred_prev'] == 'Under'])

    # Calculate the success rate
    success_rate = round(success_count / count_pred_prev_not_na, 5)

    print(f"{round(z_thresh, 1)}    {success_rate}    {success_count}")



# Write the updated dataframe to bet_result.csv
#data.to_csv('prev_perf.csv', index=False)


0.5    0.46211    250
0.6    0.46612    227
0.7    0.50614    206
0.8    0.5    186
0.9    0.48485    160
1.0    0.46263    130
1.1    0.46094    118
1.2    0.43946    98
1.3    0.42857    87
1.4    0.41379    72
1.5    0.4094    61
1.6    0.4127    52
1.7    0.41121    44
1.8    0.40964    34
1.9    0.3913    27


In [20]:

# Read the prev_perf.csv file
data = pd.read_csv('prev_perf.csv')

# Count the number of rows where pred_prev is not "NA" and is equal to PTS_Bet_Res
success_count = len(data[(data['pred_prev'] != 'NA') & (data['pred_prev'] == data['PTS_Bet_Res'])])

count_pred_prev_not_na = len(data[data['pred_prev'] == 'Under'])

# Calculate the success rate
success_rate = round(success_count / count_pred_prev_not_na, 5)

print(success_rate)


0.46263
